In [1]:
!pip install transformers==4.44.2 underthesea==6.8.4 gensim==4.3.3 scikit-learn==1.5.2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=2

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models
from underthesea import word_tokenize

In [3]:
df = pd.read_csv("/kaggle/input/cogdis-data/Combined_Data.csv")
texts = df["Original Text"].astype(str).tolist()
print(f"Loaded {len(texts)} samples")

Loaded 20092 samples


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base").to(device)
model.eval()

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [5]:
def get_phobert_embedding(text):
    # Chuẩn hóa văn bản tiếng Việt
    text = " ".join(word_tokenize(text.lower()))
    encoded = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = model(**encoded)
        last_hidden = outputs.last_hidden_state.squeeze(0)
        mask = encoded["attention_mask"].squeeze(0).unsqueeze(-1)
        mean_emb = (last_hidden * mask).sum(0) / mask.sum(0)
    return mean_emb.cpu().numpy()

embeddings = np.vstack([get_phobert_embedding(t) for t in texts])
print("✅ Embeddings shape:", embeddings.shape)

✅ Embeddings shape: (20092, 768)


In [6]:
bias_examples = {
    "All_or_Nothing": "Hoặc tôi thành công hoàn toàn, hoặc tôi là kẻ thất bại.",
    "Overgeneralization": "Tôi luôn thất bại trong mọi việc tôi làm.",
    "Mental_Filter": "Tôi chỉ thấy toàn điều tiêu cực trong cuộc sống.",
    "Disqualifying_the_Positive": "Dù người khác khen, tôi biết họ chỉ đang nói dối.",
    "Jumping_to_Conclusions": "Họ không trả lời tin nhắn, chắc chắn họ ghét tôi.",
    "Personalization": "Mọi chuyện tồi tệ đều là lỗi của tôi.",
    "Emotional_Reasoning": "Tôi cảm thấy vô vọng, nên chắc chắn mọi thứ đều tệ.",
    "Should_Statements": "Tôi phải làm tốt hơn, tôi không được phép mắc sai lầm.",
    "Labeling_and_Mislabeling": "Tôi là kẻ thất bại, kẻ vô dụng.",
    "Magnification_and_Minimization": "Chuyện nhỏ nhưng tôi thấy nó thật khủng khiếp."
}

bias_embeddings = {k: get_phobert_embedding(v) for k, v in bias_examples.items()}

def compute_similarity(vec, bias_embs):
    sims = {f"sim_{k}": cosine_similarity([vec], [v])[0][0] for k, v in bias_embs.items()}
    return sims

semantic_features = [compute_similarity(e, bias_embeddings) for e in embeddings]
semantic_df = pd.DataFrame(semantic_features)
df = pd.concat([df, semantic_df], axis=1)

In [7]:
tokenized_docs = [word_tokenize(text.lower()) for text in texts]
dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

lda = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    passes=5,
    random_state=42
)
topic_distributions = [lda.get_document_topics(bow) for bow in corpus]
df["dominant_topic"] = [max(dist, key=lambda x: x[1])[0] if dist else -1 for dist in topic_distributions]

In [8]:
topic_labels = {
    0: "Trải nghiệm cá nhân / Cảm xúc chủ quan",
    1: "Tương tác xã hội / Quan hệ giữa người với người",
    2: "Tự phản tỉnh / Nghi ngờ bản thân",
    3: "Lý giải – Mâu thuẫn nội tâm",
    4: "Tự phán xét / Trách nhiệm cá nhân"
}

df["topic_name"] = df["dominant_topic"].map(topic_labels)

In [9]:
df.head(5)

,Original Text,Label,Emotional Reasoning,Overgeneralization,Should Statements,Personalization,Mental Filter,Disqualifying the Positive,Jumping to Conclusions,Labeling and Mislabeling,...,sim_Mental_Filter,sim_Disqualifying_the_Positive,sim_Jumping_to_Conclusions,sim_Personalization,sim_Emotional_Reasoning,sim_Should_Statements,sim_Labeling_and_Mislabeling,sim_Magnification_and_Minimization,dominant_topic,topic_name
0,"Cô ấy luôn có những hành động kỳ lạ, ví dụ như...",0,0,0,0,0,0,0,0,0,...,0.628883,0.723907,0.653244,0.672899,0.690805,0.684154,0.556376,0.679951,1,Tương tác xã hội / Quan hệ giữa người với người
1,"Nếu bố mẹ tôi biết tôi đang vật lộn thế nào, h...",1,0,0,0,0,0,0,1,0,...,0.640957,0.720224,0.658066,0.681721,0.622646,0.729893,0.598981,0.640424,2,Tự phản tỉnh / Nghi ngờ bản thân
2,"Trong năm thứ 4 và thứ 5 của sự nghiệp, tôi kh...",1,0,0,0,0,0,0,0,0,...,0.686763,0.723619,0.702813,0.712168,0.730453,0.734595,0.648183,0.630595,4,Tự phán xét / Trách nhiệm cá nhân
3,"Tôi muốn bị ốm, và tôi biết điều đó thật khủng...",0,0,0,0,0,0,0,0,0,...,0.661297,0.749456,0.646059,0.714616,0.692761,0.771162,0.621756,0.753430,4,Tự phán xét / Trách nhiệm cá nhân
4,Trước khi chúng tôi bắt đầu hẹn hò và trong nă...,0,0,0,0,0,0,0,0,0,...,0.653814,0.636051,0.650368,0.608729,0.672790,0.610535,0.569708,0.595569,1,Tương tác xã hội / Quan hệ giữa người với người


In [10]:
np.save("phobert_embeddings.npy", embeddings)
df.to_csv("Semantic_features_phobert_10bias.csv", index=False)